# 비화재보 분류 
---

## 데이터 전처리
  2019~2021 신고내용 데이터 결합       
  신고내용 중 숫자와 영어같은 비화재보 분류와 관련없는 단어들 제거

In [2]:
import pandas as pd
import numpy as np 
import re

In [10]:
start_year= 2019
end_year =2021

df_all =pd.DataFrame()
#년도합치기
for year in range(start_year,end_year+1):
    globals()[f'df_{year}']=pd.DataFrame()
#Sheet 합치기
    for page in range(1,10):
        tmp = pd.read_excel(f"{year}년 전체 신고건수_관할 및 재난주소 추가.xls",sheet_name = f'Sheet {page}')['신고내용'].dropna(axis=0).T
        #결측치제거
        tmp.reset_index()
        globals()[f'df_{year}']= pd.concat([globals()[f'df_{year}'],tmp], axis = 0)
    df_all= pd.concat([df_all,globals()[f'df_{year}']], axis = 0)

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but 

In [8]:
list_tmp= list(df_all)
list_all=[]

'1'

In [9]:
for i in range(len(list_tmp)
    tmp=re.sub('\d','',list_tmp[i])
    tmp=re.sub('[A-Z]','',list_tmp[i])
    tmp=re.sub('[a-z]','',list_tmp[i])
    list_all.append(tmp)

''

In [70]:
from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer()
vect.fit(list_all)
vect.vocabulary_

{'미군': 255611,
 '캠프워크': 479733,
 '헌병대': 515157,
 '남구청': 180121,
 '교통사고': 156180,
 '추가': 471217,
 '현장': 516875,
 '경찰출동중': 144937,
 '빨리오라고': 292133,
 '하고': 503194,
 '전화끊음': 432527,
 '보성황실': 276410,
 '102동': 6633,
 '407호': 69627,
 '112': 10197,
 '동시': 222025,
 '통보': 489471,
 '구급상황센터': 157039,
 '이관': 405401,
 '수성구': 326983,
 '다리삐어': 194336,
 '거동불가': 135766,
 '병원안내': 274070,
 '삼주협화타운': 301041,
 '211동': 50887,
 '318호': 62161,
 '아버지': 348607,
 '거동불편하신데': 135855,
 '폐암환자': 498713,
 '열이': 374228,
 '오르고': 378855,
 '있다': 418183,
 '의식': 404078,
 '수면제를': 326165,
 '20알': 50522,
 '먹었다': 242874,
 '어머니': 363805,
 '의식유': 404355,
 '호흡유': 521495,
 '송현동': 324584,
 '1966': 23575,
 '24': 53227,
 '2층': 59596,
 '주택': 446393,
 '음주후': 402415,
 '수면제': 326122,
 '과다복용': 152674,
 '10알정도': 9237,
 '상담': 301781,
 '도둑이': 214732,
 '들어와': 229600,
 '112안내함': 10547,
 '금호지구': 166659,
 '사수동': 295307,
 '주취자': 446241,
 '지금여기가어딘지모른다': 451372,
 '위치': 396240,
 'gps': 118530,
 '무응답': 251626,
 '푸르지오': 500430,
 '102': 6499,
 '702호': 9